In [1]:
from pathlib import Path

from pyspark.sql.session import SparkSession
from pyspark.sql import DataFrame

MAX_MEMORY = "5g"
spark = (
    SparkSession.builder.master("local[*]")
    .appName("TripAnaliysis")
    .config("spark.excutor.memory", MAX_MEMORY)
    .config("spark.driver.memory", MAX_MEMORY)
    .getOrCreate()
)

24/02/18 18:06:45 WARN Utils: Your hostname, limhaneul-ui-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.81 instead (on interface en0)
24/02/18 18:06:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/02/18 18:06:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/18 18:06:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
import os

def read_parquet_data(year: int) -> list[DataFrame]:
    # 모든 파케이 파일 경로
    def parquet_file_all() -> list[str]:
        return sorted(list(Path(os.getcwd()).joinpath(f"data/{str(year)}").glob("*")))
    
    # return parquet_file_all()        
    return [spark.read.parquet(f"file:///{data}") for data in parquet_file_all()]

In [5]:
from pyspark.sql.functions import col
from pyspark.sql import functions as F

data = read_parquet_data(2019)[0]

In [6]:
def datetime_groupby(data: DataFrame, name: str, agg_name: str) -> DataFrame:
    return (
        data.select(F.split(col(name), " ")[0].name("pickup"))
        .groupBy("pickup")
        .agg(F.count("*").name(agg_name))
    )

In [95]:
from pyspark.sql.column import Column
from pyspark.sql import functions as F

datetime_groupby(data, "request_datetime", "pickup").toPandas().to_csv("testtt.csv")

def null_checking(value: Column) -> Column:
    return F.when(value.isNull(), value).otherwise(value)

reuqest = (
    data.select(F.split(col("request_datetime"), " ")[0].name("pickup"))
    .d
    .agg(F.count("*"))
)
reuqest.toPandas().to_csv("test.csv")
